* Ссылки 
  * [Разведочный анализ данных](https://www.kaggle.com/emstrakhov/eda-with-pandas).<br>
  * [Построение графиков](https://pcnews.ru/blogs/spargalka_po_vizualizacii_dannyh_v_python_s_pomosu_plotly-986119.html).<br>
  * [Продвинутая визуализация](https://habr.com/ru/company/skillfactory/blog/510320/).<br>
  * [Machine Learning course](https://mlcourse.ai/).<br>
  * [Фреймы в pandas](https://habr.com/ru/company/ruvds/blog/494720/)<br>
  * [Открытие фрейма в pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)<br>

In [ ]:
import numpy as np
import pandas as pd
import plotly
from IPython.display import Image
import matplotlib
from IPython.core.interactiveshell import InteractiveShell
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


InteractiveShell.ast_node_interactivity = 'all'

# Настройки показа графиков в блокноте
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 85)
pd.set_option('display.max_columns', 8)

# Рабочие фреймы
df = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')
# Объединенный список из фреймов
comb = pd.concat([df, tsd], axis=0)

#df = pd.read_csv('AmesHousing.txt', sep="\t", header = 0, encoding='cp1251', index_col=False)
#df = pd.read_csv('beverage_r.csv', sep=";", decimal=',', parse_dates=[0], index_col='numb.obs')
#df = pd.read_csv('diamond.dat', header=None, sep='\s+', names=['weight', 'price'])
#df = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)

In [ ]:
df.head().T

In [ ]:
print('Число строк и колонок в таблице: ',df.shape)
print('Типы данных в колонках:','\n',df.dtypes)

In [ ]:
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print('Колонки с числовыми значениями:', '\n',numeric_cols)

In [ ]:
df_non_numeric = df.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print('Колонки с объектными значениями:', '\n',non_numeric_cols)

In [ ]:
print('Пропущенные значения в колонках:', '\n',df.isnull().sum())

In [ ]:
print('Пропущенные значения в колонках в %:', '\n', 100 * (df.isna().sum()[df.isna().sum() > 0] / len(df)))

In [ ]:
# import cufflinks as cf
# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

# Построение гистограмм для всех колонок
def show_hist(df):
    for col in df.columns:
        df[col].iplot(kind='hist', title=col)


show_hist(df)

In [ ]:
# Выделяем числовые колонки в отдельную группу, получаем список заголовков колонок
num_feat = [x for x in comb.columns if comb[x].dtype !="object"]
num_feat.remove("Id")

# Выделяем категориальные колонки в отдельную группу
cat_feat = [x for x in comb.columns if comb[x].dtype == "object"]

In [ ]:
# Анализ пропущенных значений
# Тепловаая карта
sns.heatmap(comb[comb.columns].isnull(), cmap=sns.color_palette(
    ['#000099', '#ffff00']))  # желтый - пропущенные данные

# Матрица
missingno.matrix(comb[num_feat], figsize=(20, 4))

# График в процентах
cat_missing = comb[cat_feat].isna().sum()
cat_missing = 100 * (cat_missing[cat_missing > 0] / len(comb[cat_feat]))

plt.figure(figsize=(10, 5))
sns.barplot(x=cat_missing.sort_values(ascending=False).values,
            y=cat_missing.sort_values(ascending=False).index)
plt.title("Пропущенные значения в колонках, в %")

In [ ]:
# Статистика по колонке
pd.DataFrame(df.Data.describe())

df['data'].value_counts()

In [ ]:
#Анализ объектов в колонке. Выделение уникальных, сплит по знакам и выделение в отдельную колонку 
df['data_new'] = df.Data.apply(lambda name: name.split(',')[1].split('.')[0].strip()) 
df.Data.nunique()

In [ ]:
# График по целевой колонке
fig, ax = plt.subplots(figsize=(10,4))
sns.distplot(df['data'], bins=30, kde=True, ax=ax)

#  График логарифма
df['data'] = np.log1p(df['data'])

In [ ]:
# общий график значений по колонке и связь предиктора с откликом
fig = plt.figure(figsize=(24,15))

plt.subplot(4,3,1)
sns.distplot(comb["data"])

plt.subplot(4,3,2)
sns.scatterplot(x="data", y="y", data=comb)

In [ ]:
# Описание числовых колонок
df.drop("Id", axis=1).describe(include=['number']).T

In [ ]:
# Plot распределение предикторов во всех числовых колонках

fig = plt.figure(figsize=(20,20))

for i in range(len(num_feat)):
    plt.subplot(14,5, i+1)
    sns.distplot(comb[num_feat[i]], rug=True, hist=False, kde_kws={'bw':0.1})
    plt.title(num_feat[i])
    plt.xlabel("Value")
    plt.ylabel("Count")
    plt.tight_layout()
fig.show()

In [ ]:
# plot распределение в категориальных колонках
fig = plt.figure(figsize=(18, 30))

for i , col in enumerate(cat_feat):
    plt.subplot(12,5, i+1)
    sns.boxplot(x=col, y='y', data=df)
    plt.ylabel("Log() Y")
    plt.tight_layout()
    
fig.show()

In [ ]:
# Сведение данных к более крупным новым блокам
df['data'] = pd.cut(df.Data, 4)

In [ ]:
# Проверка уникальных значений в колонке
print(df.Data.unique())

In [ ]:
# Агрегирование. Подсчет количества записей
df.groupby('data').count()

In [ ]:
# Проверка на недостающие значения
pd.DataFrame(df.isnull().sum()).plot.line().set_title(
    "Number of missing values in the given features")
# Тепловая карта пропущенных значений
sns.heatmap(df.isnull(), cbar=False).set_title("Missing values heatmap")
print('Число строк в фрейме: {0}'.format(len(df)))
print('Число строк без пропущенных значений: {0}'.format(
    len(df.dropna(how='any'))))

In [ ]:
# Проверка на корреляцию всех числовых колонок
df.corr()
corr_matrix = df[num_feat].corr()
plt.figure(figsize=(16,12))
sns.heatmap(corr_matrix.T, annot=True, cbar=False, cmap='coolwarm');

# Отдельно выделяем все, что выше 0.8
corr_matrix = df[num_feat].corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr_matrix.T, annot=True, mask= corr_matrix < 0.8 ,cbar=False, cmap='coolwarm');

In [ ]:
#Выбор наибольших строк с наибольшими параметрами в колонке
df.nlargest(3,'data') #Обратный метод - .nsmallest

In [ ]:
# График зависимости отклика от каждого из предикторов в числовых колонках
fig = plt.figure(figsize=(20,20))

for i, col in enumerate(comb[num_feat].columns):
    plt.subplot(12,5, i+1) #Размеры графиков и их координаты на фигуре
    sns.scatterplot(x=comb[col], y=df['SalePrice']) 
    plt.tight_layout() #Автоматическое расположение графиков на фигуре
    
fig.show()

In [ ]:
#Повернем фрейм, разбив один столбец на несколько, в зависимости от значений, все объекты выровнены по дате
pivoted_df = df.pivot(index='Date', columns='Code', values='VWAP')

In [ ]:
#Метод if-then. Если условия соблюдены, то производит действия над переменными в фрейме
df.mask(df % 2 == 0, -df) #В данном случае если число четное, то знак меняется на противоположный

In [ ]:
# кросс-таблица (таблица сопряжённости)
pd.crosstab(df['data'], df['data1'])

In [ ]:
# Разбиение переменных на категории и выделение в новый столбец
df['data_new'] = pd.cut(df['data'], bins=[0, 7.90, 14.45, 31.28, 120], labels=[
                        'Low', 'Mid', 'High_Mid', 'High'])

In [ ]:
# Средние показатель выборки по определенному параметру
df[df['data'] == 1]['data1'].mean()

# Условие может быть составным
df[(df['data'] == 1) & (df['data1'] == "male")]['data2'].mean()

In [ ]:
# Выборка всех строк из фрейма по конкретному параметру в колонке
grouped = df.groupby('Data').get_group('param')

In [ ]:
# Среднее значение второго параметра для колонки фрейма, сгрупированного по первому параметру в другой колонке
income_mean = df.groupby('data1')['data2'].mean()

In [ ]:
# Выбираем из матрицы интересующую нас ячейку
first_patient = df.loc[0, 'data']

In [ ]:
# Выбираем срез из матрицы по интересующим нас колонкам
df.loc[:10, ["data1", "data2"]]

In [ ]:
# Строим график по распределению значений в колонке
(df.Data.value_counts(normalize=True) * 100).plot.barh().set_title(
    "Training Data - Percentage of people survived and Deceased")

In [ ]:
# Строим картинку
df['Survived'].value_counts().plot(kind='bar')
# Добавляем подпись
plt.title(u'Число выживших на Титанике')
plt.xticks([0, 1], ['Not Survived', 'Survived'], rotation=None)
plt.text(-0.1, 275, '61.5%', fontsize=15, color='silver')
plt.text(0.9, 150, '38.5%', fontsize=15, color='silver')

In [ ]:
# Хак для того, чтобы исправить наезжающие заголовки графика
ax.get_figure().suptitle('')

# Нужно, чтобы подписи к картинкам отображались корректно по-русски
matplotlib.rc('font', family='Arial')

# Доля выживших на Титанике
fig, ax = plt.subplots(1)
df['Survived'].value_counts().plot(kind='pie', labels=['', ''],
                                   ax=ax, autopct='%.2f%%', fontsize=15)
ax.set_title(u'Доля выживших на Титанике')
ax.axis('equal')
ax.legend(labels=[u'Выжили', u'Погибли'], framealpha=0)
ax.set_ylabel('')
plt.tight_layout()

In [ ]:
# Подробное распределение внутри класса по определенным параметрам
plt.figure(figsize=(15, 8))  # увеличим размер картинки
sns.countplot(y='data', hue='data1', data=df)

In [ ]:
# Круговая диаграмма распределения значений в колонке
df.Data.value_counts().plot.pie().legend(labels=["Class 3", "Class 1", "Class 2"],
                                         loc='center right',
                                         bbox_to_anchor=(2.25, 0.5)
                                         ).set_title("Training Data - People travelling in different classes")

In [ ]:
# График сравнения значений в колонках по дополнительному признаку из другой колонки. В процентах

data1 = round((df[df.Data == 1].Survived == 1).value_counts()
              [1]/len(df[df.Data == 1]) * 100, 2)
data2 = round((df[df.Data == 2].Survived == 1).value_counts()
              [1]/len(df[df.Data == 2]) * 100, 2)
data3 = round((df[df.Data == 3].Survived == 1).value_counts()
              [1]/len(df[df.Data == 3]) * 100, 2)
pclass_perc_df = pd.DataFrame(
    {"Percentage Survived": {"Class 1": data1, "Class 2": data2, "Class 3": data3},
     "Percentage Not Survived": {"Class 1": 100-data1, "Class 2": 100-data2, "Class 3": 100-data3}})
pclass_perc_df.plot.bar().set_title(
    "Training Data - Percentage of people survived on the basis of class")

In [ ]:
# Вариант графика сравнения, две бинарных переменных
df['data_new'] = pd.cut(df.Data, [0, 10, 20, 30, 40, 50, 60, 70, 80])
sns.countplot(x="data_new",
              hue="data1",
              data=df,
              palette=["C1", "C0"]).legend(labels=["Deceased", "Survived"]
                                           )

In [ ]:
# График распределения переменных по характеристике второй переменной
for x in [1, 2, 3]:  # задаем число классов
    df.data0[df.data1 == x].plot(kind="kde")
plt.title("Age density in classes")
plt.legend(("1st", "2nd", "3rd"))

In [ ]:
# Вариант графика сравнения, две бинарных переменных
sns.distplot(df['data'].dropna(),
             color='darkgreen',
             bins=30
             )

In [ ]:
# Вероятностная гистограмма
df['data'].hist(density=True, bins=60)
# Сравнение гистограмм
df.groupby('data')['data'].plot.hist(alpha=.6)
plt.legend()

In [ ]:
# Линейная гистограмма
ss = pd.DataFrame()
ss['survived'] = df.Data1
ss['sibling_spouse'] = pd.cut(
    df.Data2, [0, 1, 2, 3, 4, 5, 6, 7, 8], include_lowest=True)
(ss.sibling_spouse.value_counts()).plot.area().set_title(
    "Training Data - Number of siblings or spouses vs survival count")

In [ ]:
# Проверка на тип тренда (линейный или нет), тип сезонности (аддитивный или мультипликативный), его длину, выбросы
# Видим линейный тренд и мультипликативную сезонность. Это подтверждает логирафмирование цикла

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(121)
df['data'].plot(ax=ax1)
ax1.set_title(u'Объём пассажироперевозок')
ax1.set_ylabel(u'Тысяч человек')

ax2 = fig.add_subplot(122)
pd.Series(np.log10(df['data'])).plot(ax=ax2)
ax2.set_title(u'log10 от объёма пассажироперевозок')
ax2.set_ylabel(u'log10 от тысяч человек')

In [ ]:
# Быстрая проверка по интересующим нас данным. Перебор -> результат
age = df['data']
age[age < 22].shape[0]

In [ ]:
# В случае очевидного смешения двух нормальных распределений, можно оценить их более подробно
df.groupby('data')['data1'].plot.hist(alpha=0.6)
df.groupby('data')['data1'].plot.hist(density=True)  # Нормализованный вариант
plt.legend(loc='upper left')

In [ ]:
# Боксплот
sns.boxplot(x=df['data'])

In [ ]:
# Первый признак разбивает на классы, второй дает среднее значение параметра по группе
df.groupby('data')['data1'].mean().round().plot(kind='bar')
plt.ylabel('Age')  # добавляем подпись на оси Оу
plt.show()

In [ ]:
# Матрица диаграмм рассеивания: комплексное сравнение по нескольким переменным. Диагональ - ядерная оценка плотности
colors = {'genuine': 'green', 'counterfeit': 'red'}
scatter_matrix(df,
               figsize=(6, 6),  # размер картинки
               diagonal='kde',  # плотность вместо гистограммы на диагонали
               c=df['data'].replace(colors),  # цвета классов
               alpha=0.2  # степень прозрачности точек
               )

In [ ]:
# Scatter плот для исследования пар числовых признаков
plt.scatter(df['Sex'], df['Pclass'])
# Для изучения совместного распределения двух числовых признаков
sns.jointplot(x='height', y='weight', data=df)

In [ ]:
# Для исследования трёх и более признаков сводные таблицы (pivot tables)
# Index - признаки, по которым выполняется группировка. Values - признаки, по которым вычисляются значения функции
df.pivot_table(values=['age', 'cardio'], index=[
               'smoke', 'alco'], aggfunc='mean')

In [ ]:
# Анализ объектов в колонке. Выделение уникальных, сплит по знакам и выделение в отдельную колонку
df['data_new'] = df.Data.apply(lambda name: name.split(',')[
                               1].split('.')[0].strip())
df.Data.nunique()

# Построение облачного графика из объектов, где размер коррелирует с частотой
wc = WordCloud(width=1000, height=450, background_color='white').generate(
    str(df.Data_new.values))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

df.Data_new.value_counts()

In [ ]:
x = np.arange(0, 5, 0.1)


def f(x):
    return x**2


def h(x):
    return np.sin(x)


def k(x):
    return np.cos(x)


def m(x):
    return np.tan(x)


fig = go.Figure()  # создаем фигуру
fig = make_subplots(rows=2,  # Создание нескольких графиков на одной фигуре
                    cols=2,  # Число колонок на фигуре
                    column_widths=[1, 1],  # Пропорции размера графиков
                    specs=[  # Указываем расположение графиков в фигуре
                        # Указываем что первый график двойной. Чтобы объединить по гризонтали colspan
                        [{'rowspan': 2}, {}],
                        [None, {}]  # Выбрасываем нижнего соседа первого графика None
                    ]
                    )
fig.update_yaxes(range=[-0.5, 2.5],  # Зум к определенному участку на графике
                 zeroline=True,  # Рисует оси на графике
                 zerolinewidth=2,
                 zerolinecolor='LightPink',
                 col=2  # Уточнение к какой колонке применять
                 )
fig.update_xaxes(range=[-0.5, 2.5], zeroline=True,
                 zerolinewidth=2, zerolinecolor='#008000', col=2)
fig.add_trace(go.Scatter(x=x,
                         y=x**2,
                         name='f(x)=x<sup>2</sup>',  # HTML в подписях
                         mode='lines+markers'),
              1,  # Координаты на фигуре, указывающие к какому графику принадлежат
              2
              )
fig.add_trace(go.Scatter(x=x,
                         y=x,
                         name='g(x)=x',  # Метод графика с плавной линией
                         mode='markers'
                         ),
              1,
              2
              )

fig.add_trace(go.Scatter(visible='legendonly',  # Невидимый режим, показывается только при клике на легенду
                         x=x,
                         y=h(x),
                         name='x=sin(x)'),
              2,
              2
              )
fig.add_trace(go.Scatter(visible='legendonly',
                         x=x,
                         y=m(x),
                         name='x=tan(x)'),
              1,
              1
              )
fig.update_layout(legend_orientation='h',  # Смещение легенды под график
                  # Убрать отступы вокруг графика
                  margin=dict(l=0, r=0, t=30, b=0),
                  # Смещение легенды по центру
                  legend=dict(x=.5, xanchor="center"),
                  hovermode='x',  # Показывает одновременно все значения Y для точки Х
                  title='Заголовок',
                  xaxis_title='ось Х',
                  yaxis_title='ось Y'
                  )
fig.update_traces(hoverinfo='all',  # Отображается значение подписи на курсоре. Например 'x+y'
                  hovertemplate='Аргумент: %{x}<br>Функция: %{y}'
                  )
fig.update_xaxes(title='Ось Х графика 1', col=1, row=1)
fig.update_yaxes(title='Ось Y графика 1', col=1, row=1)
fig.update_xaxes(title='Ось Х графика 2', col=2, row=1)
fig.update_yaxes(title='Ось Y графика 2', col=2, row=1)
fig.update_xaxes(title='Ось Х графика 3', col=2, row=2)
fig.update_yaxes(title='Ось Y графика 3', col=2, row=2)

fig.show()

In [ ]:
# Вставляем изображения в блокнот
display(Image(url='https://habrastorage.org/webt/9m/2c/zd/9m2czdt-uv7oe6v-nws3frtw7-a.jpeg', width = 200))

In [ ]:
#Описываем все колонки: тип, заполненность, охват значений, значения предикторов